In [1]:
import os
import re
import shutil
import pandas as pd
import numpy as np

In [2]:
'''
In this folder, there are test target of each prompt.
index 0 is the string of test target, e.g. "I want to ..."
index 1 is the ground truth.
index 2 is the accessibility type.
'''
path_target = "dataset\\RQ2_dataset\\task\\"

'''
In this folder, there are prompts of each test.
Using this folder to get the initial prompt.
'''
path_prompt = "dataset\\RQ2_dataset\\prompt\\"

'''
This folder saves the result of LLM, and the source of statistic analysis.
'''
path_result = "result\\"

#You may want to filter files which only have this string to exclude files you do not want in path_result.
file_str = "result"

#Statistic analysis result is saved in this file.
result_file_str = "statistic\\accuracy_QWen.xlsx"

#Match the choice, e.g. "1: Click ..."
re_choice = re.compile(r'\d+: ') 

In [4]:
#Write the head of the file.
with open(result_file_str, "w", encoding = "UTF-8") as f:
    f.write("group_name\ttest_id\ttest_case\tright_answer\tacce_widget\tacce_GUI\tmodify\tlen_of_prompt\tnum_of_choice")
    for i in range(0, 12):
        f.write("\t" + str(i))
    for i in range(0, 12):
        f.write("\t" + str(i))
    f.write("\n")

#Get the list of score, and write the score to the result file.
files = os.listdir(path_result)
for file in files:
    #Filter files which are not relevant to result.
    if file.find(file_str) == -1:
        continue
    if file.find(".csv") == -1:
        continue
    print(file)
        
    np_file = np.array(pd.read_csv(path_result + "\\" + file, index_col = 0))
    #Note that the filename of score and prompt is similar, we replace the name.
    file_target = file.replace(file_str, "task", 1)
    file_target = file_target.replace("__", "_")
    #The structure of np_target has been introduced. Seeing in path_target.
    
    #Making file_target, getting the file.
    match_integer = re.compile("\d+")
    indexs = re.finditer(match_integer, file_target)
    span_list = []
    test_index = 0
    for index in indexs:
        span_list.append(index.span())
        test_index = int(index.group())
    target_span = span_list[-1]
    file_target = file_target[0: target_span[0]] + ".csv"

    np_target = np.array(pd.read_csv(path_target + "\\" + file_target))
    with open(result_file_str, "a+", encoding = "UTF-8") as f:
        #Write front lines.
        f.write(str(file)) #Name
        f.write("\t")
        f.write(str(test_index + 1)) #Id
        f.write("\t")
        for j in range(0, 3):
            f.write(str(np_target[test_index][j]))# Target, ground_truth, accessibility type
            f.write('\t')
        #accessibility of this page.
        np_accessibility = np.zeros(5)
        for j in range(0, len(np_target)):
            np_accessibility[int(np_target[j][2])] = np_accessibility[int(np_target[j][2])] + 1
        acce_page = 0
        acce_max = 0
        for j in range(1, 5):
            if np_accessibility[j] > acce_max:
                acce_max = np_accessibility[j]
                acce_page = j
        f.write(str(acce_page))
        f.write("\t")
        
        std_answer = np_target[test_index][1]
        
        if file_target.find("acce") != -1 or file_target.find("disa") != -1:# If this GUI is modified
            f.write("1")
        else:
            f.write("0")
        f.write("\t")

        dir_this_target = file.replace(file_str, "LLM_prompt", 1).replace("__", "_").replace(".csv", "", 1) + ".txt"
        dir_this_target = path_prompt + dir_this_target

        #Length of the choice.
        std_choice = 0
        with open(dir_this_target, "r", encoding = "UTF-8") as this_target:
            content = this_target.read()
            f.write(str(len(content.split())))
            f.write("\t")

            choice_list = re_choice.findall(content)
            count_choice = 0
            for choice in choice_list:
                count_choice = count_choice + 1
            f.write(str(count_choice))
            f.write("\t")
            std_choice = count_choice
            
        #Calculate the hit ratio and stability of LLM result.
        np_result = np.zeros(len(np_file[0]))
        for i in range(0, len(np_file)):
            for j in range(0, len(np_file[i])):
                if np_file[i][j] == std_answer:
                    np_result[j] = np_result[j] + 0.2
        #np_result is the hit ratio.

        #Translate np_file to calculate the stability.
        np_file = np_file.T
        np_stability = np.zeros(len(np_file))
        for i in range(0, len(np_file)):
            #Firstly, calculate the number appears the most.
            np_appear_time = np.zeros((len(np_file[i]), 2))
            for j in range(0, len(np_file[i])):
                if np_file[i][j] != 0:
                    match_pos = 0
                    while np_appear_time[match_pos][1] != 0:
                        if np_appear_time[match_pos][0] == np_file[i][j]:
                            break
                        match_pos = match_pos + 1
                    np_appear_time[match_pos][0] = np_file[i][j]
                    np_appear_time[match_pos][1] = np_appear_time[match_pos][1] + 1
            appear_max_sum = 0
            appear_max_amount = 0
            appear_max_time = 0
            for j in range(0, len(np_appear_time)):
                if np_appear_time[j][0] == 0:
                    continue
                if np_appear_time[j][1] > appear_max_time:
                    appear_max_time = np_appear_time[j][1]
                    appear_max_sum = np_appear_time[j][0]
                    appear_max_amount = 1
                elif np_appear_time[j][1] == appear_max_time:
                    appear_max_sum = appear_max_sum + np_appear_time[j][0]
                    appear_max_amount = appear_max_amount + 1
            if appear_max_amount != 0:
                appear_most = appear_max_sum / appear_max_amount
            else:
                appear_most = 1
            #Secondly, calculate the stability.
            for j in range(0, len(np_file[i])):
                if np_file[i][j] == 0:
                    np_stability[i] = np_stability[i] + std_choice ** 2
                else:
                    np_stability[i] = np_stability[i] + (np_file[i][j] - appear_most) ** 2
            np_stability[i] = ((np_stability[i] / len(np_file[i])) ** 0.5) / std_choice

        #Write to result file.
        for i in range(0, len(np_result)):
            f.write(str(np_result[i]))
            f.write("\t")
        for i in range(0, len(np_stability)):
            f.write(str(np.around(np_stability[i], decimals = 4)))
            if i < len(np_stability) - 1:
                f.write("\t")
            else:
                f.write("\n")

resultBlueMoon_1__0.csv
resultBlueMoon_1__1.csv
resultBlueMoon_1__2.csv
resultBlueMoon_2__0.csv
resultBlueMoon_2__1.csv
resultBlueMoon_2__2.csv
resultBlueMoon_2__3.csv
resultBlueMoon_2__4.csv
resultBlueMoon_2__5.csv
resultBlueMoon_3__0.csv
resultBlueMoon_3__1.csv
resultBlueMoon_3__2.csv
resultBlueMoon_3__3.csv
resultBlueMoon_3__4.csv
resultBlueMoon_3__5.csv
resultBlueMoon_3__6.csv
resultBlueMoon_3__7.csv
resultBlueMoon_3__8.csv
resultBlueMoon_4__0.csv
resultBlueMoon_4__1.csv
resultBlueMoon_4__2.csv
resultBlueMoon_4__3.csv
resultBlueMoon_4__4.csv
resultBlueMoon_4__5.csv
resultBlueMoon_4__6.csv
resultBlueMoon_5__0.csv
resultBlueMoon_5__1.csv
resultBlueMoon_5__10.csv
resultBlueMoon_5__11.csv
resultBlueMoon_5__12.csv
resultBlueMoon_5__13.csv
resultBlueMoon_5__2.csv
resultBlueMoon_5__3.csv
resultBlueMoon_5__4.csv
resultBlueMoon_5__5.csv
resultBlueMoon_5__6.csv
resultBlueMoon_5__7.csv
resultBlueMoon_5__8.csv
resultBlueMoon_5__9.csv
resultchrono_1__0.csv
resultchrono_1__1.csv
resultchrono_1__